In [ ]:
{
 "cells": [
  {
   "cell_type": "code",
   "execution_count": 0,
   "metadata": {
    "application/vnd.databricks.v1+cell": {
     "cellMetadata": {},
     "inputWidgets": {},
     "nuid": "ef1abaff-f125-4489-85ec-fbeaa88fdf1b",
     "showTitle": false,
     "tableResultSettingsMap": {},
     "title": ""
    }
   },
   "outputs": [],
   "source": [
    "# Databricks notebook source\n",
    "import requests\n",
    "import pandas as pd\n",
    "from datetime import datetime\n",
    "\n",
    "# API configuration\n",
    "CITY = \"Hanoi\"\n",
    "URL = \"https://api.open-meteo.com/v1/forecast?latitude=21.0285&longitude=105.8542&current_weather=true\"\n",
    "\n",
    "# Fetch data from API\n",
    "try:\n",
    "    response = requests.get(URL)\n",
    "    response.raise_for_status()  # Raise exception for HTTP errors\n",
    "    data = response.json()\n",
    "    \n",
    "    # Extract current weather data\n",
    "    current_weather = data.get(\"current_weather\", {})\n",
    "    \n",
    "    # Create DataFrame with the new API response structure\n",
    "    weather_data = {\n",
    "        \"city\": [CITY],\n",
    "        \"timestamp\": [datetime.now().isoformat()],\n",
    "        \"api_timestamp\": [current_weather.get(\"time\")],\n",
    "        \"temperature\": [current_weather.get(\"temperature\")],\n",
    "        \"windspeed\": [current_weather.get(\"windspeed\")],\n",
    "        \"winddirection\": [current_weather.get(\"winddirection\")],\n",
    "        \"is_day\": [current_weather.get(\"is_day\")],\n",
    "        \"weathercode\": [current_weather.get(\"weathercode\")],\n",
    "        \"latitude\": [data.get(\"latitude\")],\n",
    "        \"longitude\": [data.get(\"longitude\")],\n",
    "        \"elevation\": [data.get(\"elevation\")]\n",
    "    }\n",
    "\n",
    "    df = pd.DataFrame(weather_data)\n",
    "    \n",
    "    # Save as temporary Delta table\n",
    "    spark_df = spark.createDataFrame(df)\n",
    "    spark_df.write.mode(\"overwrite\").saveAsTable(\"temp_weather_raw\")\n",
    "    \n",
    "    dbutils.notebook.exit(\"Successfully fetched weather data for \" + CITY)\n",
    "\n",
    "except requests.exceptions.RequestException as e:\n",
    "    dbutils.notebook.exit(f\"API request failed: {str(e)}\")\n",
    "except Exception as e:\n",
    "    dbutils.notebook.exit(f\"An error occurred: {str(e)}\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 0,
   "metadata": {
    "application/vnd.databricks.v1+cell": {
     "cellMetadata": {},
     "inputWidgets": {},
     "nuid": "845ae0f1-8a0a-40d2-8341-07155b8ecdab",
     "showTitle": false,
     "tableResultSettingsMap": {},
     "title": ""
    }
   },
   "outputs": [],
   "source": [
    "# Databricks notebook source\n",
    "from pyspark.sql.functions import col, from_unixtime, unix_timestamp, lit\n",
    "from pyspark.sql.functions import when\n",
    "\n",
    "# Read raw data\n",
    "raw_df = spark.table(\"temp_weather_raw\")\n",
    "\n",
    "# Transform data\n",
    "transformed_df = raw_df.withColumn(\n",
    "    \"processed_timestamp\", \n",
    "    from_unixtime(unix_timestamp(col(\"timestamp\")))\n",
    ").withColumn(\n",
    "    \"weather_condition\",\n",
    "    # Map WMO weather codes to human-readable descriptions\n",
    "    when(col(\"weathercode\") == 0, \"Clear sky\")\n",
    "     .when(col(\"weathercode\").isin([1, 2, 3]), \"Partly cloudy\")\n",
    "     .when(col(\"weathercode\").isin([45, 48]), \"Fog\")\n",
    "     .when(col(\"weathercode\").isin([51, 53, 55]), \"Drizzle\")\n",
    "     .when(col(\"weathercode\").isin([56, 57]), \"Freezing drizzle\")\n",
    "     .when(col(\"weathercode\").isin([61, 63, 65]), \"Rain\")\n",
    "     .when(col(\"weathercode\").isin([66, 67]), \"Freezing rain\")\n",
    "     .when(col(\"weathercode\").isin([71, 73, 75]), \"Snow fall\")\n",
    "     .when(col(\"weathercode\").isin([77]), \"Snow grains\")\n",
    "     .when(col(\"weathercode\").isin([80, 81, 82]), \"Rain showers\")\n",
    "     .when(col(\"weathercode\").isin([85, 86]), \"Snow showers\")\n",
    "     .when(col(\"weathercode\") == 95, \"Thunderstorm\")\n",
    "     .when(col(\"weathercode\").isin([96, 99]), \"Thunderstorm with hail\")\n",
    "     .otherwise(\"Unknown\")\n",
    ").select(\n",
    "    \"city\",\n",
    "    \"processed_timestamp\",\n",
    "    \"temperature\",\n",
    "    \"windspeed\",\n",
    "    \"winddirection\",\n",
    "    \"is_day\",\n",
    "    \"weather_condition\",\n",
    "    \"weathercode\",\n",
    "    \"latitude\",\n",
    "    \"longitude\"\n",
    ")\n",
    "\n",
    "transformed_df.display()\n",
    "\n",
    "# Create temp view for next task\n",
    "transformed_df.write.mode(\"overwrite\").saveAsTable(\"temp_weather_transformed\")\n",
    "\n",
    "# COMMAND ----------\n",
    "dbutils.notebook.exit(\"Data transformation completed successfully\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 0,
   "metadata": {
    "application/vnd.databricks.v1+cell": {
     "cellMetadata": {},
     "inputWidgets": {},
     "nuid": "1666eb4b-7c79-48c1-971f-390664e9da7c",
     "showTitle": false,
     "tableResultSettingsMap": {},
     "title": ""
    }
   },
   "outputs": [],
   "source": [
    "# Databricks notebook source\n",
    "from datetime import datetime\n",
    "\n",
    "# Read transformed data\n",
    "transformed_df = spark.table(\"temp_weather_transformed\")\n",
    "\n",
    "# Define output path with timestamp\n",
    "output_path = f\"/mnt/weather_data/hanoi/{datetime.now().strftime('%Y/%m/%d/%H%M')}\"\n",
    "\n",
    "# Save as Parquet\n",
    "transformed_df.write.parquet(output_path, mode=\"overwrite\")\n",
    "\n",
    "# COMMAND ----------\n",
    "dbutils.notebook.exit(f\"Weather data saved to DBFS at {output_path}\")"
   ]
  }
 ],
 "metadata": {
  "application/vnd.databricks.v1+notebook": {
   "computePreferences": null,
   "dashboards": [],
   "environmentMetadata": {
    "base_environment": "",
    "environment_version": "2"
   },
   "inputWidgetPreferences": null,
   "language": "python",
   "notebookMetadata": {
    "pythonIndentUnit": 4
   },
   "notebookName": "Deploy Workloads with Databricks Workflows",
   "widgets": {}
  },
  "language_info": {
   "name": "python"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 0
}